In [1]:
%matplotlib inline
from __future__ import division
import torch

from counterfactual_xai.utils.lsat_dataloader.lsat_dataloader import LsatDataloader
from counterfactual_xai.utils.datafeed import DataFeed
from counterfactual_xai.utils.clue.gaussian_mlp import GaussianMLP
from counterfactual_xai.utils.clue.bnn.gaussian_bnn import GaussianBNN
from counterfactual_xai.utils.clue.bnn.train import train_BNN_regression

CSV_PATH = ""
INPUT_DIMS = [1, 1, 8, 2]

x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds, DATA_KEYS, input_dims = LsatDataloader(INPUT_DIMS,
                                                                                                           CSV_PATH).get_lsat_dataset()

trainset = DataFeed(x_train, y_train, transform=None)
valset = DataFeed(x_test, y_test, transform=None)

y_means = torch.Tensor(y_means)
y_stds = torch.Tensor(y_stds)

input_dim = x_train.shape[1]
width = 200
depth = 2
output_dim = y_train.shape[1]
model = GaussianMLP(input_dim, width, depth, output_dim, flatten_image=False)

N_train = x_train.shape[0]
batch_size = 512
# nb_epochs = 2200 # We can do less iterations as this method has faster convergence
nb_epochs = 2200
log_interval = 1

lr = 1e-2

## weight saving parameters #######
burn_in = 120  # this is in epochs 
sim_steps = 20  # We want less correlated samples -> despite having per minibatch noise we see correlations
N_saves = 100

resample_its = 10
resample_prior_its = 50  # 45 can be choosen to better control overfitting 
re_burn = 1e7

cuda = torch.cuda.is_available()
net = GaussianBNN(model, N_train, lr=lr, cuda=cuda)

save_dir = "./results/"

cost_train, cost_dev, rms_dev, ll_dev = train_BNN_regression(net, save_dir, batch_size, nb_epochs, trainset, valset,
                                                             cuda,
                                                             burn_in, sim_steps, N_saves, resample_its,
                                                             resample_prior_its,
                                                             re_burn, flat_ims=False, nb_its_dev=10, y_mu=y_means,
                                                             y_std=y_stds)



INFO:counterfactual_xai.utils.lsat_dataloader.lsat_dataloader:Formatting Dataframes


Total params: 0.04M
Init cost variables:


/Users/lukasscholz/repositorys/counterfactuals/counterfactual_xai/utils/clue/bnn/stochastic_gradient_hamilton_sampler.py:80: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/python_arg_parser.cpp:1630.)
  d_p.add_(weight_decay, p.data)


it 0/2200, Jtr_pred = 1.409488, it 1/2200, Jtr_pred = 1.351609, it 2/2200, Jtr_pred = 1.348958, it 3/2200, Jtr_pred = 1.348597, it 4/2200, Jtr_pred = 1.347855, it 5/2200, Jtr_pred = 1.343386, it 6/2200, Jtr_pred = 1.345929, it 7/2200, Jtr_pred = 1.354947, it 8/2200, Jtr_pred = 1.344402, it 9/2200, Jtr_pred = 1.344058, it 10/2200, Jtr_pred = 1.343259, it 11/2200, Jtr_pred = 1.348125, it 12/2200, Jtr_pred = 1.346650, it 13/2200, Jtr_pred = 1.349946, it 14/2200, Jtr_pred = 1.345492, it 15/2200, Jtr_pred = 1.347671, it 16/2200, Jtr_pred = 1.345094, it 17/2200, Jtr_pred = 1.343143, it 18/2200, Jtr_pred = 1.349923, it 19/2200, Jtr_pred = 1.346336, it 20/2200, Jtr_pred = 1.342309, it 21/2200, Jtr_pred = 1.348282, it 22/2200, Jtr_pred = 1.342767, it 23/2200, Jtr_pred = 1.345634, it 24/2200, Jtr_pred = 1.343228, it 25/2200, Jtr_pred = 1.346224, it 26/2200, Jtr_pred = 1.344229, it 27/2200, Jtr_pred = 1.347687, it 28/2200, Jtr_pred = 1.343659, it 29/2200, Jtr_pred = 1.350407, it 30/2200, Jtr_pred

KeyboardInterrupt: 